In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
from PIL import Image as PILImage
from langchain_openai import ChatOpenAI
from langgraph.constants import START, END
from langgraph.graph import MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import RemoveMessage, HumanMessage, SystemMessage

# Load environment variables
load_dotenv()
base_model = os.getenv("BASE_MODEL", "gpt-4o")  # Default model if not set
llm = ChatOpenAI(model=base_model, max_retries=2, max_tokens=80, temperature=1)

# Define state structure
class ConversationState(MessagesState):
    summary: str = ""  # Initialize summary as empty


# Function to handle conversation
def call_model(state: ConversationState):
    """
    Process the current conversation state by invoking the language model.
    """
    summary = state.summary

    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state.messages
    else:
        messages = state.messages

    response = llm.invoke(messages)
    return {"messages": state.messages + [response]}


# Function to summarize conversation
def summarize_conversation(state: ConversationState) -> ConversationState:
    """
    Generate or update a summary of the current conversation.
    """
    summary = state.summary

    if summary:
        summary_message = (
            f"This is a summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state.messages + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)

    delete_messages = [RemoveMessage(id=m.id) for m in state.messages[:-2]]

    return {"summary": response.content, "messages": delete_messages}


# Function to determine conversation flow
def should_continue(state: ConversationState):
    """
    Decide whether to continue the conversation or summarize it.
    """
    if len(state.messages) > 6:
        return "summarize_conversation"
    return END


# In-memory configuration (no persistent storage)
config = {"configurable": {"thread_id": "session_1"}}

# Initialize the workflow
workflow = StateGraph(ConversationState)
workflow.add_node("conversation", call_model)
workflow.add_node("summarize_conversation", summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Use memory-based checkpointing (no persistence)
memory = MemorySaver()
llm_agent = workflow.compile(checkpointer=memory)

# Display graph
image_data = llm_agent.get_graph().draw_mermaid_png()

# Ensure resources directory exists
resources_dir = Path("resources")
resources_dir.mkdir(parents=True, exist_ok=True)

graph_image_path = resources_dir / "graph.png"

with open(graph_image_path, "wb") as f:
    f.write(image_data)

img = PILImage.open(graph_image_path)
img.show()

# Start interactive chat session
conversation_history = []  # Store session conversation in memory

while True:
    user_query = input("\nYou: ")
    
    if user_query.lower() == "exit":
        print("Ending session.")
        break

    elif user_query == "get_prev":
        # Retrieve previous messages stored in memory
        if not conversation_history:
            print("\n[No previous messages found]")
        else:
            print("\n[Previous Messages]")
            for msg in conversation_history:
                print(f"{msg.role.capitalize()}: {msg.content}")

    else:
        # Process user input and track conversation
        message = HumanMessage(content=user_query)
        output = llm_agent.invoke({"messages": conversation_history + [message]}, config)
        
        # Store conversation history
        conversation_history = output["messages"]
        
        # Display latest response
        print(f"\nAI: {conversation_history[-1].content}")


Opening in existing browser session.


AttributeError: 'dict' object has no attribute 'summary'